In [3]:
import meld_graph
import meld_graph.models
import meld_graph.experiment
import meld_graph.dataset
import meld_graph.data_preprocessing
import meld_graph.evaluation


import importlib
importlib.reload(meld_graph)
importlib.reload(meld_graph.models)
importlib.reload(meld_graph.dataset)
importlib.reload(meld_graph.experiment)
importlib.reload(meld_graph.data_preprocessing)
importlib.reload(meld_graph.evaluation)

import logging
import os
import json

from meld_graph.dataset import GraphDataset, Oversampler
from meld_classifier.meld_cohort import MeldCohort, MeldSubject
from meld_graph.training import Metrics
import numpy as np
from meld_graph.paths import EXPERIMENT_PATH

from meld_graph.evaluation import Evaluator



Setting MELD_DATA_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting BASE_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments/co-ripa1/
Setting FS_SUBJECTS_PATH to 
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1


## Define models 

In [29]:
# initialise models you want to run
EXPERIMENT_PATH='/rds/project/kw350/rds-kw350-meld/experiments_graph/kw350'

model_base_paths = {
   'dcd': '/rds/project/kw350/rds-kw350-meld/experiments_graph/kw350/22-12-13_finetune/dcd/fold_00/',
   'dc': '/rds/project/kw350/rds-kw350-meld/experiments_graph/kw350/22-12-13_finetune/dc/fold_00/',
   'dcd_head': '/rds/project/kw350/rds-kw350-meld/experiments_graph/kw350/22-12-13_finetune/dcd_head/fold_00/',
}

### Create dataset with vals data

In [5]:
model_base_path=model_base_paths['dcd']
checkpoint_path=os.path.join(EXPERIMENT_PATH, model_base_path)
exp = meld_graph.experiment.Experiment.from_folder(checkpoint_path)

subjects = exp.data_parameters['val_ids']

cohort= MeldCohort(hdf5_file_root='{site_code}_{group}_featurematrix_combat_6.hdf5', dataset='MELD_dataset_V6.csv')

features= exp.data_parameters['features']

# initiate params
params = {                
          "features":features,
          "augment_data": {}, 
          "lesion_bias":0,
          "lobes": False,
          "synthetic_data": {
              'run_synthetic': False,
            },
          "number_of_folds": 10,
          "preprocessing_parameters": {
              "scaling": None, 
              "zscore": '../data/feature_means.json',
            },
          "combine_hemis": None,
          }

#load dataset
dataset = GraphDataset(subjects, cohort, params, mode='test')



Initialised Experiment 22-12-13_finetune/dcd
Loading and preprocessing test data


dataset using distance_maps


Z-scoring data for MELD_H17_3T_FCD_0092
Z-scoring data for MELD_H5_3T_C_0005
Z-scoring data for MELD_H2_15T_C_0002
Z-scoring data for MELD_H4_15T_FCD_0005
Z-scoring data for MELD_H4_3T_FCD_0008
Z-scoring data for MELD_H16_3T_FCD_042
Z-scoring data for MELD_H26_15T_FCD_0006
Z-scoring data for MELD_H4_3T_FCD_0041
Z-scoring data for MELD_H17_3T_FCD_0069
Z-scoring data for MELD_H14_3T_FCD_0028
Z-scoring data for MELD_H3_3T_C_0029
Z-scoring data for MELD_H3_3T_C_0025
Z-scoring data for MELD_H17_3T_FCD_0129
Z-scoring data for MELD_H21_3T_FCD_0022
Z-scoring data for MELD_H15_3T_C_0017
Z-scoring data for MELD_H4_15T_FCD_0022
Z-scoring data for MELD_H24_3T_FCD_0009
Z-scoring data for MELD_H12_3T_FCD_0029
Z-scoring data for MELD_H3_3T_C_0022
Z-scoring data for MELD_H10_3T_FCD_0006
Z-scoring data for MELD_H17_3T_FCD_0037
Z-scoring data for MELD_H3_3T_C_0027
Z-scoring data for MELD_H5_3T_FCD_0016
Z-scoring data for MELD_H17_3T_FCD_0063
Z-scoring data for MELD_H5_3T_FCD_0023
Z-scoring data for MELD

In [7]:
len(dataset.subject_ids)

45

### Load models and predict

In [27]:
use_preload_dataset = True 
output_path = '/rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1/22-12-13_evaluation_real_valsdata'

In [28]:
from pyexpat import model

if use_preload_dataset:    
    cohort = None
    subjects = None
else:
    dataset = None

for model_name in model_base_paths.keys(): 
    
    #load experiment already trained using checkpoint path
    model_base_path=model_base_paths[model_name]
    checkpoint_path=os.path.join(EXPERIMENT_PATH, model_base_path)
    exp = meld_graph.experiment.Experiment.from_folder(checkpoint_path)

    # Run the evaluation on the test data and save into directory provided
    save_dir=os.path.join(output_path, model_name)

    eva = Evaluator(experiment = exp,
                    checkpoint_path = checkpoint_path,
                    save_dir = save_dir ,
                    make_images = True,
                    dataset=dataset,
                    cohort=cohort,
                    subject_ids = subjects,
                    mode = 'test'
                )

    # evaluate (predict , stats and plot) or just run individually each step
    # eva.evaluate()
    
    # # load data and predict
    eva.load_predict_data()
    # # calculate stats 
    eva.stat_subjects()
    #  # make images 
    eva.plot_subjects_prediction()

    

Initialised Experiment 22-12-13_finetune/dcd_head
Creating model
Loading model weights from checkpoint /rds/project/kw350/rds-kw350-meld/experiments_graph/kw350/22-12-13_finetune/dcd_head/fold_00/best_model.pt
loading data and predicting model
saving prediction for MELD_H17_3T_FCD_0092
saving distance_map for MELD_H17_3T_FCD_0092
saving prediction for MELD_H5_3T_C_0005
saving distance_map for MELD_H5_3T_C_0005
saving prediction for MELD_H2_15T_C_0002
saving distance_map for MELD_H2_15T_C_0002
saving prediction for MELD_H4_15T_FCD_0005
saving distance_map for MELD_H4_15T_FCD_0005
saving prediction for MELD_H4_3T_FCD_0008
saving distance_map for MELD_H4_3T_FCD_0008
saving prediction for MELD_H16_3T_FCD_042
saving distance_map for MELD_H16_3T_FCD_042
saving prediction for MELD_H26_15T_FCD_0006
saving distance_map for MELD_H26_15T_FCD_0006
saving prediction for MELD_H4_3T_FCD_0041
saving distance_map for MELD_H4_3T_FCD_0041
saving prediction for MELD_H17_3T_FCD_0069
saving distance_map for

### Compare models

In [30]:
# load results 
import pandas as pd

df={}
predicted={}
for model_name in model_base_paths.keys():  
    
    results_file = os.path.join(output_path,model_name, 'results', 'test_results.csv')
    df[model_name] = pd.read_csv(results_file) 
    
    

In [13]:
df[model_name].keys()

Index(['ID', 'group', 'detected', 'tp', 'fp', 'fn', 'tn', 'dice lesional',
       'dice non-lesional'],
      dtype='object')

In [33]:
# stats
import pandas as pd

for model_name in model_base_paths.keys():  
   
    dfsub = df[model_name]    

    print(f'Model {model_name}: ')
    
    mean_dice_les = dfsub[dfsub['group']==True]['dice lesional'].mean()
    print(f'mean dice lesional patient : {mean_dice_les}')

    mean_dice_les_all = dfsub['dice lesional'].mean()
    print(f'mean dice lesional all : {mean_dice_les_all}')

    mean_dice_nonles_all = dfsub['dice non-lesional'].mean()
    print(f'mean dice non lesional all : {mean_dice_nonles_all}')

    sensitivity = len(dfsub[(dfsub['group']==True) & (dfsub['tp']>0)])/len(dfsub[(dfsub['group']==True)])
    print(f'sensitivity patient : {sensitivity}')

    specificity = len(dfsub[(dfsub['group']==False) & (dfsub['fp']==0)])/len(dfsub[(dfsub['group']==False)])
    print(f'specificity controls : {specificity}')

    print(f'\n')

Model dcd: 
mean dice lesional patient : 0.076969637953125
mean dice lesional all : 0.34362285365555556
mean dice non lesional all : 0.9958684039999999
sensitivity patient : 0.28125
specificity controls : 1.0


Model dc: 
mean dice lesional patient : 0.13545142465624999
mean dice lesional all : 0.3629876797555555
mean dice non lesional all : 0.9959633835555555
sensitivity patient : 0.3125
specificity controls : 0.9230769230769231


Model dcd_head: 
mean dice lesional patient : 0.12087076096875
mean dice lesional all : 0.3748414300222222
mean dice non lesional all : 0.995927800888889
sensitivity patient : 0.375
specificity controls : 1.0


